Dlib Recognition Test

In [1]:
import cv2
import mediapipe as mp
import face_recognition
import pickle
import time
import psutil
import os

# === Load known face encodings ===
with open("known_faces_dlib.pkl", "rb") as f:
    known_faces = pickle.load(f)

known_encodings = []
known_names = []
for name, enc_list in known_faces.items():
    for enc in enc_list:
        known_encodings.append(enc)
        known_names.append(name)

# === Initialize MediaPipe Face Detection ===
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# === Webcam ===
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Failed to open webcam.")
    exit()

# === Benchmark Variables ===
start_time = time.time()
frame_count = 0
total_faces_detected = 0
cpu_usages = []

# === Timing Limit ===
DURATION = 1000  # seconds

print("🎥 Running real-time recognition for 10 seconds...")

while cap.isOpened():
    now = time.time()
    if now - start_time > DURATION:
        break

    success, frame = cap.read()
    if not success:
        break

    frame_count += 1
    cpu_usages.append(psutil.cpu_percent(interval=None))

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb)

    if results.detections:
        total_faces_detected += len(results.detections)

        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x = int(bbox.xmin * w)
            y = int(bbox.ymin * h)
            width = int(bbox.width * w)
            height = int(bbox.height * h)

            x = max(0, x)
            y = max(0, y)
            width = min(w - x, width)
            height = min(h - y, height)

            face_location = (y, x + width, y + height, x)

            name = "Unknown"
            encodings = face_recognition.face_encodings(rgb, known_face_locations=[face_location])
            if encodings:
                encoding = encodings[0]
                matches = face_recognition.compare_faces(known_encodings, encoding)
                face_distances = face_recognition.face_distance(known_encodings, encoding)
                if any(matches):
                    best_match_index = face_distances.argmin()
                    name = known_names[best_match_index]

            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(frame, name, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # === Display FPS and CPU Usage ===
    elapsed = now - start_time
    fps = frame_count / elapsed if elapsed > 0 else 0
    current_cpu = cpu_usages[-1] if cpu_usages else 0

    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.putText(frame, f"CPU: {current_cpu:.1f}%", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 100, 100), 2)

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

# === Results ===
end_time = time.time()
avg_fps = frame_count / (end_time - start_time)
avg_cpu = sum(cpu_usages) / len(cpu_usages) if cpu_usages else 0

print("\n📊 Benchmark Results:")
print(f"🕒 Runtime: {end_time - start_time:.2f} seconds")
print(f"🖼️ Frames processed: {frame_count}")
print(f"📈 Average FPS: {avg_fps:.2f}")
print(f"👥 Total faces detected: {total_faces_detected}")
print(f"🧠 Average CPU usage: {avg_cpu:.2f}%")

cap.release()
cv2.destroyAllWindows()


2025-04-16 17:39:25.307216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744814365.323041  133572 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744814365.327676  133572 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744814365.339864  133572 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744814365.339879  133572 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744814365.339880  133572 computation_placer.cc:177] computation placer alr

Please install `face_recognition_models` with this command before using `face_recognition`:

pip install git+https://github.com/ageitgey/face_recognition_models


NameError: name 'quit' is not defined

Arcface Recognition Test

In [2]:
import cv2
import insightface
import pickle
import numpy as np
import os
import time
import psutil

# === Suppress logs ===
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["INSIGHTFACE_LOG_LEVEL"] = "ERROR"

# === Load known embeddings ===
with open("known_faces_arcface.pkl", "rb") as f:
    raw_faces = pickle.load(f)

known_names = []
known_embeddings = []

for name, embeddings in raw_faces.items():
    for emb in embeddings:
        norm_emb = emb / np.linalg.norm(emb)
        known_names.append(name)
        known_embeddings.append(norm_emb)

# === Init InsightFace ===
model = insightface.app.FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=0)  # GPU=0, CPU=-1

# === Webcam feed ===
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Failed to open webcam.")
    exit()

# === Constants and Benchmark Vars ===
DURATION = 1000  # seconds
frame_count = 0
total_faces_detected = 0
cpu_usages = []

print("🎥 Starting ArcFace Recognition for 10 seconds...")

# Wait for first frame before starting timer
while True:
    ret, frame = cap.read()
    if ret:
        break
print("✅ First frame acquired, starting timer...")
start_time = time.time()

# === Main Loop ===
while cap.isOpened():
    now = time.time()
    if now - start_time > DURATION:
        break

    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    cpu_usages.append(psutil.cpu_percent(interval=None))

    faces = model.get(frame)

    if faces:
        total_faces_detected += len(faces)

    for face in faces:
        x1, y1, x2, y2 = face.bbox.astype(int)
        embedding = face.embedding / np.linalg.norm(face.embedding)

        name = "Unknown"
        similarities = [np.dot(embedding, k) for k in known_embeddings]
        best_index = int(np.argmax(similarities))
        best_score = similarities[best_index]

        if best_score > 0.36:
            name = known_names[best_index]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, name, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # === Show FPS and CPU on screen ===
    elapsed = now - start_time
    fps = frame_count / elapsed if elapsed > 0 else 0
    current_cpu = cpu_usages[-1] if cpu_usages else 0

    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.putText(frame, f"CPU: {current_cpu:.1f}%", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 100, 100), 2)

    cv2.imshow("ArcFace Recognition", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

# === Summary ===
end_time = time.time()
avg_fps = frame_count / (end_time - start_time)
avg_cpu = sum(cpu_usages) / len(cpu_usages) if cpu_usages else 0

print("\n📊 Benchmark Results:")
print(f"🕒 Runtime: {end_time - start_time:.2f} seconds")
print(f"🖼️ Frames processed: {frame_count}")
print(f"📈 Average FPS: {avg_fps:.2f}")
print(f"👥 Total faces detected: {total_faces_detected}")
print(f"🧠 Average CPU usage: {avg_cpu:.2f}%")

cap.release()
cv2.destroyAllWindows()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/maksimgorki/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3,

Deepface Recogniiton Test

In [3]:
import cv2
import pickle
import numpy as np
from deepface import DeepFace
import time
import psutil

# === Settings ===
ENCODING_FILE = "known_faces_deepface.pkl"
MODEL_NAME = "ArcFace"
DETECTOR_BACKEND = "mediapipe"
DURATION = 10  # seconds

# === Load known embeddings ===
print("📂 Loading known embeddings...")
with open(ENCODING_FILE, "rb") as f:
    known_faces = pickle.load(f)

# === Webcam feed ===
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Failed to open webcam.")
    exit()

print("🎥 Running DeepFace recognition for 10 seconds...")

# Wait for first frame before starting timer
while True:
    ret, frame = cap.read()
    if ret:
        break
print("✅ First frame acquired, starting timer...")
start_time = time.time()

# === Benchmarking Variables ===
frame_count = 0
total_faces_detected = 0
cpu_usages = []

while cap.isOpened():
    now = time.time()
    if now - start_time > DURATION:
        break

    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    cpu_usages.append(psutil.cpu_percent(interval=None))

    try:
        detected_faces = DeepFace.extract_faces(
            img_path=frame,
            detector_backend=DETECTOR_BACKEND,
            enforce_detection=False
        )

        total_faces_detected += len(detected_faces)

        for face_obj in detected_faces:
            face_img = face_obj.get("face")
            facial_area = face_obj.get("facial_area")

            if face_img is None or facial_area is None:
                continue

            x = facial_area.get("x", 0)
            y = facial_area.get("y", 0)
            w = facial_area.get("w", 0)
            h = facial_area.get("h", 0)

            rep = DeepFace.represent(
                img_path=face_img,
                model_name=MODEL_NAME,
                enforce_detection=False
            )[0]["embedding"]
            rep = np.array(rep)

            best_match = "Unknown"
            best_score = -1

            for name, enc_list in known_faces.items():
                for known_rep in enc_list:
                    known_rep = np.array(known_rep)
                    sim = np.dot(rep, known_rep) / (np.linalg.norm(rep) * np.linalg.norm(known_rep))
                    if sim > best_score:
                        best_score = sim
                        best_match = name

            label = best_match if best_score > 0.36 else "Unknown"

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    except Exception as e:
        print(f"[!] Error: {e}")

    # === Overlay FPS and CPU ===
    elapsed = now - start_time
    fps = frame_count / elapsed if elapsed > 0 else 0
    current_cpu = cpu_usages[-1] if cpu_usages else 0

    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.putText(frame, f"CPU: {current_cpu:.1f}%", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 100, 100), 2)

    cv2.imshow("DeepFace Recognition", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

# === Results ===
end_time = time.time()
avg_fps = frame_count / (end_time - start_time)
avg_cpu = sum(cpu_usages) / len(cpu_usages) if cpu_usages else 0

print("\n📊 Benchmark Results:")
print(f"🕒 Runtime: {end_time - start_time:.2f} seconds")
print(f"🖼️ Frames processed: {frame_count}")
print(f"📈 Average FPS: {avg_fps:.2f}")
print(f"👥 Total faces detected: {total_faces_detected}")
print(f"🧠 Average CPU usage: {avg_cpu:.2f}%")

cap.release()
cv2.destroyAllWindows()


AttributeError: module 'inspect' has no attribute 'ArgSpec'